In [1]:
#necesary libraries
import layoutparser.ocr as ocr
import cv2
import numpy as np
import io
import pandas as pd
import os
from PIL import Image
import pytesseract as pt
import matplotlib.pyplot as plt

In [2]:
img_path = 'C:/Users/stanl/OneDrive/Desktop/OCR/'

In [3]:
#create image path to access the different files in the folder
create_path = lambda f: os.path.join(img_path,f )

In [4]:
image_files = os.listdir(img_path)
for f in image_files:
    print(f)

.ipynb_checkpoints
files
img1.jpg
menu1.jpg
menu2.jpg
menu3.png
OCR and Pytorch.ipynb
OCR Second Attempt.ipynb
ocr-t
oct_pytess.py


In [5]:
# Set the path for the tesseract command in pytesseract to the specified location.
pt.pytesseract.tesseract_cmd = r'C:\Program files\Tesseract-OCR\tesseract.exe'

### Language Configuration

In [6]:
#using cmd: tesseract --list-langs
avb_langs = pt.get_languages(config ='')

for lang in avb_langs:
    print(lang)

chi_sim
chi_tra
eng
fra
osd
pol
rus
spa
spa_old


### Image Preprocessing Steps

In [66]:
image = cv2.imread(image_files[5])
size = (900,900)
image = cv2.resize(image, size)

In [67]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Image",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [68]:
# Gaussian Blur/Noise Removal
blur = cv2.GaussianBlur(gray, (7,7), 0)

In [69]:
cv2.imshow("Image",blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [70]:
#Otsu thresholding 
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow("Image",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
#Dilation and Kernels
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 13))
cv2.imshow("Image",kernel)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [72]:
#dilation 
dilate = cv2.dilate(thresh, kernel, iterations=1)
cv2.imshow("Image",dilate)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Bounding Boxes and Contours 

In [73]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [74]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [75]:
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [88]:
results = []
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:
        roi = image[x:x+w, y:y+h]
        cv2.rectangle(image, (x,y), (x+w, y+h), (36, 255, 12),2)
        ocr_result = pt.image_to_string(image)
        ocr_result = ocr_result.split("\n")
#         print(ocr_result)
        for item in ocr_result:
            results.append(item)
               
# cv2.imshow("Image",image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [90]:
print(results)

['<< Sangtien Brito', '', 'Hors D’Oeuvres', '', 'Free-Form Millet Tart su', '', 'Salmon Tartare: $14', 'Grilled Sea Scallops si3', 'Cheesy Papillons sn', '', 'Gruyere and Camembert Fondue $4', 'Cosmic Walnut Salad $10', '', 'Entrées', '', 'Steak Frites $20', 'Quiche Lorraine $22', 'Croque Madame sI6', '‘Croque Monsieur si6', 'Lamb Chops with Fantastique Sauce $30', 'Cassoulet gio', 'Cog Au Vin $22', 'Roasted Figs and Ham $26', '', 'Ala carte menu available daily from 5 p.m.to ll pm.', '', 'Open every day of the year. 18% gratuity on parties of 5 or more.', '', '', '<< Sangtien Brito', '', 'Hors D’Oeuvres', '', 'Free-Form Millet Tart su', '', 'Salmon Tartare: $14', 'Grilled Sea Scallops si3', 'Cheesy Papillons sn', '', 'Gruyere and Camembert Fondue $4', 'Cosmic Walnut Salad $10', '', 'Entrées', '', 'Steak Frites $20', 'Quiche Lorraine $22', 'Croque Madame sI6', '‘Croque Monsieur si6', 'Lamb Chops with Fantastique Sauce $30', 'Cassoulet gio', 'Cog Au Vin $22', 'Roasted Figs and Ham $26',

In [111]:
entities = results

In [112]:
print(entities)

['<< Sangtien Brito', '', 'Hors D’Oeuvres', '', 'Free-Form Millet Tart su', '', 'Salmon Tartare: $14', 'Grilled Sea Scallops si3', 'Cheesy Papillons sn', '', 'Gruyere and Camembert Fondue $4', 'Cosmic Walnut Salad $10', '', 'Entrées', '', 'Steak Frites $20', 'Quiche Lorraine $22', 'Croque Madame sI6', '‘Croque Monsieur si6', 'Lamb Chops with Fantastique Sauce $30', 'Cassoulet gio', 'Cog Au Vin $22', 'Roasted Figs and Ham $26', '', 'Ala carte menu available daily from 5 p.m.to ll pm.', '', 'Open every day of the year. 18% gratuity on parties of 5 or more.', '', '', '<< Sangtien Brito', '', 'Hors D’Oeuvres', '', 'Free-Form Millet Tart su', '', 'Salmon Tartare: $14', 'Grilled Sea Scallops si3', 'Cheesy Papillons sn', '', 'Gruyere and Camembert Fondue $4', 'Cosmic Walnut Salad $10', '', 'Entrées', '', 'Steak Frites $20', 'Quiche Lorraine $22', 'Croque Madame sI6', '‘Croque Monsieur si6', 'Lamb Chops with Fantastique Sauce $30', 'Cassoulet gio', 'Cog Au Vin $22', 'Roasted Figs and Ham $26',

In [113]:
entities = list(set(entities))

In [114]:
print(entities)

['', 'Quiche Lorraine $22', 'Open every day of the year. 18% gratuity on parties of 5 or more.', '‘Croque Monsieur si6', 'Cosmic Walnut Salad $10', 'Cheesy Papillons sn', 'Ala carte menu available daily from 5 p.m.to ll pm.', 'Salmon Tartare: $14', 'Entrées', 'Roasted Figs and Ham $26', 'Gruyere and Camembert Fondue $4', 'Steak Frites $20', 'Grilled Sea Scallops si3', 'Free-Form Millet Tart su', 'Cassoulet gio', 'Lamb Chops with Fantastique Sauce $30', 'Cog Au Vin $22', 'Hors D’Oeuvres', '<< Sangtien Brito', 'Croque Madame sI6']


In [115]:
entities.sort()

In [116]:
print(entities)

['', '<< Sangtien Brito', 'Ala carte menu available daily from 5 p.m.to ll pm.', 'Cassoulet gio', 'Cheesy Papillons sn', 'Cog Au Vin $22', 'Cosmic Walnut Salad $10', 'Croque Madame sI6', 'Entrées', 'Free-Form Millet Tart su', 'Grilled Sea Scallops si3', 'Gruyere and Camembert Fondue $4', 'Hors D’Oeuvres', 'Lamb Chops with Fantastique Sauce $30', 'Open every day of the year. 18% gratuity on parties of 5 or more.', 'Quiche Lorraine $22', 'Roasted Figs and Ham $26', 'Salmon Tartare: $14', 'Steak Frites $20', '‘Croque Monsieur si6']


In [118]:
entities =str(entities)

In [119]:
#using textblob to translate from french to english
from textblob import TextBlob
tb = TextBlob(entities)
translated = tb.translate(from_lang = 'fr', to='en') #customize to desired language 
print(translated)

trans_text = str(translated)
# print(type(trans_text))

['', '<< Brato Brito', 'Ala Menu Menu Avairable Daily from 5 p.m.to ll pm.', 'Cassoulet Gio', 'Cheesy Papillons Sn', 'Cog au vin $ 22', 'Cosmic Walnut Salad $ 10' , 'Croque Madame Si6', 'Entrances', 'Free-Form Millet Tart SU', 'Grilled Sea Scallops Si3', 'Gruyere and Camembert Fondue $ 4', 'out of works', 'Lamb chops with fantastic sauce $ 30' , 'Open Every Day of the Year. 18% free on part of 5 or more. ',' Quiche lorraine $ 22 ',' roasted figs and ham $ 26 ',' salmon tartare: $ 14 ',' steak fries $ 20 ',' ‘Croque Monsieur Si6 ']


In [128]:
import json

with open('output.json', 'w') as json_outfile:
    json.dump(trans_text, json_outfile)